In [1]:
#r "nuget:XPlot.Plotly"
#r "nuget:Deedle"
#r "nuget:DotNetZip"
#r "nuget:NodaTime"

**Type real password here**

In [2]:
let password = ""

Installing package XPlot.Plotly.done!

Installing package Deedle.done!

Installing package DotNetZip.done!

Installing package NodaTime.done!

In [3]:
let allLinesSource = 
    [ 
        ("BadLines",       [ 490; 675; 677; 681; 671 ])
        ("GoodLines",      [ 783; 834; 782; 837; 785 ])
        ("InnerCityLines", [ 403; 505; 216; 906; 302 ])
    ]
let singleLineSource folder lineName = [ (folder, [ lineName ]) ]    

**Choose source lines to load**

In [4]:
//let sources = singleLineSource "BadLines" 490
let sources = allLinesSource

**Choose stops to work with**

In [5]:
//let stops = [ "40870" ]
let stops = []

**Specify desired daily time periods**

In [6]:
open NodaTime

let rec getPeriod t od = function
    | [] -> "Unknown"
    | (p, check)::tail when check t od -> p
    | _::tail -> getPeriod t od tail

let inPeriod from' to' (t:ZonedDateTime) (od:LocalDate) = t.Date = od && from' <= t.Hour && t.Hour < to'
let periods = 
    [ 
        ("06:00 - 10:00", (inPeriod 6  10))
        ("10:00 - 14:00", (inPeriod 10 14))
        ("14:00 - 18:00", (inPeriod 14 18))
        ("18:00 - 22:00", (inPeriod 18 22))
        ("Other", (inPeriod 0 24))
    ]

In [7]:
open Deedle

module Frame =
    let groupByType<'T, 'O when 'T : equality and 'O : equality> c (f:Frame<'O, string>) =
        let result : Frame<'T * 'O, string> = f |> Frame.groupRowsBy c 
        result

In [60]:
open Ionic.Zip
open System.IO
open System
open System.Globalization
open NodaTime.Text

let tz = DateTimeZoneProviders.Tzdb.GetZoneOrNull "Australia/Melbourne"

let LoadData folder lineName stops = 
    let sw = System.Diagnostics.Stopwatch.StartNew ()
    let findStop =
        match stops with 
        | [] -> fun _ -> true
        | _  -> fun x -> stops |> List.tryFind (fun s -> s = x) |> Option.isSome

    let zip = ZipFile.Read(sprintf @"Data/Samples/%s/Departures-%s.zip" folder lineName)
    zip.set_Password password
    use stream = zip.Entries |> Seq.head |> fun e -> e.OpenReader ()

    let f = 
        Frame.ReadCsv(stream)
        |> Frame.filterRows (fun i s -> String.IsNullOrEmpty (s.GetAs<string> "ScheduledArrivalTime") |> not)
        |> Frame.filterRows (fun i s -> s.GetAs<string> "IrsStopCode" |> string |> findStop)
    
    f?Deviation <- 
        f.GetColumn<string> "ArrivalTime"
        |> Series.zipInner (f.GetColumn<string> "ScheduledArrivalTime" ) 
        |> Series.mapValues (fun (sa, aa) -> InstantPattern.General.Parse sa, InstantPattern.General.Parse aa)
        |> Series.mapValues (fun (sa, aa) -> sa.Value, aa.Value)
        |> Series.mapValues (fun (sa, aa) -> aa - sa |> fun ts -> ts.TotalSeconds)
    
    let f =
        f
        |> Frame.filterRows (fun i s -> abs s?Deviation <= 21000.0)
        
    f?OperatingDay <- 
        f.Columns.Get("OperatingDay").As<DateTime>() 
        |> Series.mapValues (fun x -> LocalDate(x.Year, x.Month, x.Day))
    f?ScheduledTimeZoned <- 
        f.GetColumn<string> "ScheduledArrivalTime"
        |> Series.mapValues (InstantPattern.General.Parse)
        |> Series.mapValues (fun x -> x.Value.InZone tz)
    f?DayOfWeek <-
        f.GetColumn<LocalDate> "OperatingDay"
        |> Series.mapValues (fun x -> x.DayOfWeek)
    f?Period <-
        f.GetColumn<ZonedDateTime>("ScheduledTimeZoned")
        |> Series.zipInner (f.GetColumn<LocalDate> "OperatingDay" ) 
        |> Series.mapValues (fun (od, x) -> getPeriod x od periods)
    
    let f = 
        f
        |> Frame.groupByType<string, _> "Period"
        |> Frame.groupByType<IsoDayOfWeek, _> "DayOfWeek"
        |> Frame.mapRowKeys Pair.flatten3
    
    display (sprintf "Successfully loaded data for line %s/%s: %d rows in %d ms" folder lineName f.RowCount sw.ElapsedMilliseconds)
    
    f

**This will load all data as specified by code above**

In [61]:
let allFrames =
    sources
    |> Seq.collect (fun (folder, lineNames) -> lineNames |> Seq.map (fun lineName -> folder, string lineName))
    |> Seq.toArray
    |> Seq.map (fun (folder, lineName) -> lineName, ((LoadData folder lineName stops), lineName, folder))
    |> dict
    
display ("All Loaded!")

Successfully loaded data for line BadLines/490: 12650 rows in 1406 ms

Successfully loaded data for line BadLines/675: 12106 rows in 1347 ms

Successfully loaded data for line BadLines/677: 27945 rows in 3004 ms

Successfully loaded data for line BadLines/681: 40625 rows in 4585 ms

Successfully loaded data for line BadLines/671: 53337 rows in 5977 ms

...

Successfully loaded data for line InnerCityLines/505: 39910 rows in 4661 ms

Successfully loaded data for line InnerCityLines/216: 453326 rows in 50716 ms

Successfully loaded data for line InnerCityLines/906: 423401 rows in 48046 ms

Successfully loaded data for line InnerCityLines/302: 246012 rows in 30153 ms

All Loaded!

In [62]:
type StatsValues = {
    Min    : float
    Max    : float
    Mean   : float
    Median : float
    StdDev : float
    NumberOfValues : float
    
    LineName  : string
    Folder    : string
    DayOfWeek : string
    Period    : string
}

let CalcStats (series:Series<_,_>) = 
    { StatsValues.Min    = series |> Stats.min    |> Math.Round
      StatsValues.Max    = series |> Stats.max    |> Math.Round
      StatsValues.Mean   = series |> Stats.mean   |> Math.Round      
      StatsValues.Median = series |> Stats.median |> Math.Round
      StatsValues.StdDev = series |> Stats.stdDev |> Math.Round
      StatsValues.NumberOfValues = float series.KeyCount
      LineName   = ""
      Folder     = ""
      DayOfWeek  = ""
      Period     = ""
    }

**This will calculate overall statistical measurements on each loaded line**

In [63]:
let allStats = 
    allFrames.Values
    |> Seq.map (fun (f, lineName, folder) -> { CalcStats (f?Deviation) with LineName = lineName; Folder = folder })
    |> Seq.sortBy (fun s -> s.StdDev)
    |> Seq.toArray

display (allStats)
let allStatsFrame = Frame.ofRecords allStats

index,Min,Max,Mean,Median,StdDev,NumberOfValues,LineName,Folder,DayOfWeek,Period
0,-402,1751,55,33,140,47724,837,GoodLines,,
1,-287,1985,115,91,146,27945,677,BadLines,,
2,-392,2154,-42,-60,148,2353,403,InnerCityLines,,
3,-210,1568,104,79,169,12106,675,BadLines,,
4,-646,1566,54,27,172,39910,505,InnerCityLines,,
5,-1229,3827,67,45,193,246012,302,InnerCityLines,,
6,-4800,4781,104,57,214,193035,834,GoodLines,,
7,-1460,5701,74,46,216,423401,906,InnerCityLines,,
8,-745,3159,161,119,225,141735,785,GoodLines,,
9,-858,1139,51,74,228,12650,490,BadLines,,


In [64]:
let CalcBuckets series =
    let stats = CalcStats series 
    let bucketsNumber = 100.0
    let bucketSize = (stats.Max - stats.Min) / bucketsNumber

    let getBucket _ (v:int) = (float v - stats.Min) / bucketSize |> int

    let distribution = series |> Series.groupInto getBucket (fun b s -> s.KeyCount)
    let maxBucket = distribution.Max()
    bucketsNumber, bucketSize, maxBucket


In [65]:
open XPlot.Plotly

let MakePlot series forWhat w h =
    let title = sprintf "Deviation from Schedule (sec) for %s" forWhat
    let stats = CalcStats series
    let _, bucketSize, maxBucket = CalcBuckets series

    display (stats)

    let hist = 
        Histogram(
            x = series.Values, 
            xbins   = Xbins(start = stats.Min, ``end`` = stats.Max, size = bucketSize), 
            marker  = Marker(color = "yellow", line = Line(color = "gray", width = 1)),
            opacity = 0.75, 
            name = "Distribution"
        )

    let zero   = Scatter ( name = "Zero",   x = [ 0; 0 ], y = [ 0.0; maxBucket ])
    let mean   = Scatter ( name = "Mean",   x = [ stats.Mean; stats.Mean ],     y = [ 0.0; maxBucket ])
    let median = Scatter ( name = "Median", x = [ stats.Median; stats.Median ], y = [ 0.0; maxBucket ])

    let stdDev = 
        Scatter(
            x = [ stats.Mean-stats.StdDev; stats.Mean-stats.StdDev; stats.Mean+stats.StdDev; stats.Mean+stats.StdDev ],
            y = [ maxBucket; 0.0; 0.0; maxBucket ],
            name = "StdDev"
        )

    let traces = [ hist :> Trace; mean :> Trace; median :> Trace; stdDev :> Trace; zero :> Trace ]

    let plot = 
        traces
        |> Chart.Plot
        |> Chart.WithXTitle "Deviation"
        |> Chart.WithYTitle "Numner of arrivals"
        |> Chart.WithTitle title
        |> Chart.WithWidth w
        |> Chart.WithHeight h
    plot


In [66]:
let GetFrame lineName =
    let frame, lineName2, folder = 
        if String.IsNullOrEmpty lineName 
            then allFrames.Values |> Seq.head
            else allFrames.[lineName]

    let title = sprintf "%s, %s" lineName2 folder;

    frame, title

In [67]:
let CreatePlot (f:Frame<_,_>) title = MakePlot (f.Columns.Get("Deviation").As<int>()) title 800 600    

**The following will display a distribution chart for the first loaded line.**

In [68]:
display (GetFrame "" ||> CreatePlot)

Min,Max,Mean,Median,StdDev,NumberOfValues,LineName,Folder,DayOfWeek,Period
-858,1139,51,74,228,12650,,,,


In [69]:
let PrepareFrame predicates f title =
    let rec prepare f title = function
        | [] -> f, title
        | (predicate, v)::tail -> 
            let f = f |> Frame.filterRows (fun i s -> predicate s)
            let title = sprintf "%s, %s" title v
            prepare f title tail    
    prepare f title predicates
    
let Compare c (v:'T) =
    (fun (s:ObjectSeries<_>) -> s.GetAs<'T> c = v), (string v)

**The following will display a distribution chart for the first loaded line, filtered down to the desired day-of-week and daily timeframe.**

In [70]:
GetFrame ""
||> PrepareFrame 
    [Compare "DayOfWeek" IsoDayOfWeek.Monday
     Compare "Period"    "06:00 - 10:00"]
||> CreatePlot
|> display

Min,Max,Mean,Median,StdDev,NumberOfValues,LineName,Folder,DayOfWeek,Period
-431,1139,158,116,275,767,,,,


**The following code resamples the source data into (Day-Of-Week x Period) buckets.**

In [71]:
let resampledStats =
    allFrames.Values
    |> Seq.collect (fun (f, lineName, folder) -> 
        let sw = System.Diagnostics.Stopwatch.StartNew ()
        let result = 
            f?Deviation
            |> Series.sortByKey 
            |> Series.resampleEquiv (fun (dow, p, _) -> dow, p) 
            |> Series.map (fun (dow, p) s -> { CalcStats s with LineName = lineName; Folder = folder; DayOfWeek = string dow; Period = p })
            |> fun x -> x.Values
            |> Seq.where (fun s -> s.Period <> "Other")
            |> Seq.where (fun s -> s.Period <> "Unknown")
        display (sprintf "Resampled %s/%s in %d ms" folder lineName sw.ElapsedMilliseconds )
        result)
    |> Seq.toArray

Resampled BadLines/490 in 60 ms

Resampled BadLines/675 in 40 ms

Resampled BadLines/677 in 103 ms

Resampled BadLines/681 in 150 ms

Resampled BadLines/671 in 183 ms

...

Resampled InnerCityLines/403 in 6 ms

Resampled InnerCityLines/505 in 176 ms

Resampled InnerCityLines/216 in 2014 ms

Resampled InnerCityLines/906 in 1944 ms

Resampled InnerCityLines/302 in 1149 ms

In [72]:
display (resampledStats)
    
let resampledStatsFrame = resampledStats |> Frame.ofRecords

index,Min,Max,Mean,Median,StdDev,NumberOfValues,LineName,Folder,DayOfWeek,Period
0,-431,1139,158,116,275,767,490,BadLines,Monday,06:00 - 10:00
1,-483,705,126,152,238,442,490,BadLines,Monday,10:00 - 14:00
2,-706,619,57,92,220,916,490,BadLines,Monday,14:00 - 18:00
3,-547,436,-58,-25,191,243,490,BadLines,Monday,18:00 - 22:00
4,-541,531,15,51,182,854,490,BadLines,Tuesday,06:00 - 10:00
5,-487,446,19,68,199,445,490,BadLines,Tuesday,10:00 - 14:00
6,-858,803,75,82,284,1030,490,BadLines,Tuesday,14:00 - 18:00
7,-555,692,-48,-40,238,239,490,BadLines,Tuesday,18:00 - 22:00
8,-533,693,57,90,177,736,490,BadLines,Wednesday,06:00 - 10:00
9,-479,749,74,122,227,420,490,BadLines,Wednesday,10:00 - 14:00


# What line has the smallest actual arrival time distribution ? What is it ? 

**Top 10 buckets with smallest StdDev(Deviation):**

In [73]:
let top10Smallest = 
    resampledStats
    |> Seq.sortBy (fun x -> x.StdDev)
    |> Seq.truncate 20
    |> Seq.toArray

display(top10Smallest)

index,Min,Max,Mean,Median,StdDev,NumberOfValues,LineName,Folder,DayOfWeek,Period
0,-215,-2,-80,-69,55,24,681,BadLines,Sunday,18:00 - 22:00
1,-135,234,67,62,61,286,677,BadLines,Saturday,06:00 - 10:00
2,-159,251,32,19,80,536,837,GoodLines,Sunday,06:00 - 10:00
3,-225,348,42,30,82,2143,834,GoodLines,Saturday,06:00 - 10:00
4,-188,456,86,76,84,1985,671,BadLines,Saturday,14:00 - 18:00
5,-145,474,110,100,85,2239,671,BadLines,Saturday,10:00 - 14:00
6,-171,458,101,91,89,2190,671,BadLines,Friday,10:00 - 14:00
7,-163,529,82,71,90,814,671,BadLines,Saturday,06:00 - 10:00
8,-144,479,73,56,91,1239,671,BadLines,Monday,18:00 - 22:00
9,-203,411,6,-6,91,1563,837,GoodLines,Monday,18:00 - 22:00


# What line has the largest actual arrival time distribution? What is it?

**Top 10 buckets with largest StdDev(Deviation):**

In [74]:
let top10Largest = 
    resampledStats
    |> Seq.sortByDescending (fun x -> x.StdDev)
    |> Seq.truncate 20
    |> Seq.toArray

display(top10Largest)

index,Min,Max,Mean,Median,StdDev,NumberOfValues,LineName,Folder,DayOfWeek,Period
0,-173,3815,223,93,598,3572,671,BadLines,Friday,14:00 - 18:00
1,-2006,4781,237,98,546,7972,834,GoodLines,Friday,14:00 - 18:00
2,-701,6602,122,41,532,15109,216,InnerCityLines,Monday,18:00 - 22:00
3,-281,3880,194,93,483,3227,671,BadLines,Thursday,14:00 - 18:00
4,-708,6691,141,52,446,15445,216,InnerCityLines,Friday,14:00 - 18:00
5,-2964,2207,209,117,428,1668,681,BadLines,Thursday,14:00 - 18:00
6,-738,2869,186,71,420,15528,216,InnerCityLines,Thursday,14:00 - 18:00
7,-237,2154,43,-38,415,87,403,InnerCityLines,Wednesday,14:00 - 18:00
8,-3584,2857,159,73,396,2163,681,BadLines,Friday,14:00 - 18:00
9,-222,3159,320,235,376,5003,785,GoodLines,Tuesday,10:00 - 14:00


# What is the mean distribution per day per 4 hours block? (for all lines, 06:00-10:00 / 10:00-14:00 / 14:00-18:00 / 18:00-22:00)

**Don't quite know how to answer this. Need clarification**

In [75]:
let MakeStatsPlot w h (maxx, maxy, minx, miny) subTitle hc vc gc tc getColor (f:Frame<int, string>) =
    let title = sprintf "Stats %s (%s)" vc hc
    let initx = [ minx; maxx ]
    let inity = [ miny; maxy ]
    let dots = 
        f.Rows
        |> Series.mapValues (fun s -> 
            let x    = s.GetAs<float> hc
            let y    = s.GetAs<float> vc
            let key  = s.GetAs<string> gc
            let text = s.GetAs<string> tc
            key, ((x, y), text))
        |> fun x -> x.Values
        |> Seq.toArray

    let extreme get sort = dots |> Seq.map (fun (_, (xy, _)) -> get xy) |> Seq.sortBy id |> sort |> Seq.tryHead |> Option.defaultValue 0.0

    let maxx = extreme fst id
    let maxy = extreme snd id
    let minx = extreme fst Seq.rev
    let miny = extreme snd Seq.rev

    let traces = 
        dots
        |> Seq.groupBy fst
        |> Seq.map (fun (key, s) -> 
            Scatter ( 
                name = key,  
                x    = (s |> Seq.map (snd >> fst >> fst) |> Seq.toArray),
                y    = (s |> Seq.map (snd >> fst >> snd) |> Seq.toArray),
                text = (s |> Seq.map (snd >> snd) |> Seq.toArray),
                mode = "markers", marker = Marker ( color = getColor key )))
        |> Seq.toList
        |> fun x -> (Scatter (x = initx, y = inity, mode = "markers", name = "aux", marker = Marker (color = "white")))::x

    let plot = 
        traces
        |> Chart.Plot
        |> Chart.WithXTitle hc
        |> Chart.WithYTitle vc
        |> Chart.WithTitle (sprintf "%s - %s" subTitle title)
        |> Chart.WithWidth w
        |> Chart.WithHeight h

    plot, (maxx, maxy, minx, miny)


In [76]:
let colorByDow = function
    | "Monday"    -> "red" 
    | "Tuesday"   -> "orange" 
    | "Wednesday" -> "yellow" 
    | "Thursday"  -> "green" 
    | "Friday"    -> "cyan" 
    | "Saturday"  -> "blue" 
    | "Sunday"    -> "violet" 
    | _ -> "black"

let colorByPeriod = function
    | "06:00 - 10:00" -> "red" 
    | "10:00 - 14:00" -> "yellow" 
    | "14:00 - 18:00" -> "blue" 
    | "18:00 - 22:00" -> "violet" 
    | _ -> "black"
    
let colorByFolder = function
    | "GoodLines"      -> "blue"
    | "BadLines"       -> "red"
    | "InnerCityLines" -> "yellow"
    | _ -> "black"

let MakePlots title hc vc colorBy splitBy tc getColor =
    let MakeStatsPlot =  MakeStatsPlot 800 600
    let plot, mm = 
        resampledStatsFrame 
        |> MakeStatsPlot (0.0, 0.0, 0.0, 0.0) title hc vc colorBy tc getColor  
    display(plot);

    resampledStatsFrame.GetColumn<string> splitBy
    |> fun x -> x.Values
    |> Seq.distinct
    |> Seq.map (fun tf -> 
        resampledStatsFrame 
        |> Frame.filterRows (fun i s -> s.GetAs<string> splitBy = tf)
        |> MakeStatsPlot mm tf hc vc colorBy tc getColor)
    |> Seq.map fst
    |> Seq.toArray
    |> Array.iter (display >> ignore)
   

**The following cells visualize resampled statistics in various ways**

In [77]:
MakePlots "All Timeframes" "Mean" "StdDev" "DayOfWeek" "Period" "LineName"  colorByDow

In [78]:
MakePlots "All Days Of Week" "Mean" "StdDev" "Period" "DayOfWeek" "LineName"  colorByPeriod

In [79]:
MakePlots "All Timeframes" "Mean" "StdDev" "Folder" "Period" "LineName"  colorByFolder

In [80]:
MakePlots "All Days Of Week" "Mean" "StdDev" "Folder" "DayOfWeek" "LineName"  colorByFolder